# Altering Tables

### Prudence When Altering Tables

In [1]:
mysqldump --user='russell' -p 
rookery birds > /tmp/birds.sql;

ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'mysqldump --user='russell' -p rookery birds > /tmp/birds.sql' at line 1


In [3]:
mysqldump --user='russell' -p 
rookery > rookery.sql;

ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'mysqldump --user='russell' -p rookery > rookery.sql' at line 1


### Essential Changes

In [4]:
ALTER TABLE table_name changes;

ERROR 1046 (3D000): No database selected


In [6]:
SHOW DATABASES;

Database birdwatchers information_schema mysql performance_schema rookery test

In [12]:
USE rookery;

In [13]:
SHOW TABLES;

Tables_in_rookery bird_families bird_orders birds

In [14]:
ALTER TABLE bird_families
ADD COLUMN order_id INT;

In [15]:
CREATE TABLE test.birds_new LIKE birds;

In [17]:
USE test;
DESCRIBE birds_new;

Field Type Null Key Default Extra bird_id int(11) NO PRI NULL auto_increment scientific_name varchar(255) YES UNI NULL common_name varchar(50) YES NULL family_id int(11) YES NULL description text YES NULL

In [18]:
INSERT INTO birds_new
SELECT * FROM rookery.birds;

In [19]:
CREATE TABLE birds_new_alternative
SELECT * FROM rookery.birds;

In [20]:
DROP TABLE birds_new_alternative;

In [21]:
ALTER TABLE birds_new
ADD COLUMN wing_id CHAR(2);

In [22]:
DESCRIBE birds_new;

Field Type Null Key Default Extra bird_id int(11) NO PRI NULL auto_increment scientific_name varchar(255) YES UNI NULL common_name varchar(50) YES NULL family_id int(11) YES NULL description text YES NULL wing_id char(2) YES NULL

In [23]:
ALTER TABLE birds_new
DROP COLUMN wing_id;

In [24]:
ALTER TABLE birds_new
ADD COLUMN wing_id CHAR(2) AFTER family_id;

In [25]:
ALTER TABLE birds_new
ADD COLUMN body_id CHAR(2) AFTER wing_id,
ADD COLUMN bill_id CHAR(2) AFTER body_id,
ADD COLUMN endangered BIT DEFAULT b'1' AFTER bill_id,
CHANGE COLUMN common_name common_name VARCHAR(255);

In [26]:
UPDATE birds_new SET endangered = 0
WHERE bird_id IN(1,2,4,5);

In [27]:
SELECT bird_id, scientific_name, common_name
FROM birds_new
WHERE endangered;

bird_id scientific_name common_name 3 Aix sponsa Wood Duck 6 Apteryx mantelli North Island Brown Kiwi

In [29]:
SELECT * FROM birds_new
WHERE NOT endangered;

bird_id scientific_name common_name family_id wing_id body_id bill_id endangered description 1 Charadrius vociferus Killdeer NULL NULL NULL NULL NULL 2 Gavia immer Great Northern Loon NULL NULL NULL NULL NULL 4 Chordeiles minor Common Nighthawk NULL NULL NULL NULL NULL 5 Sitta carolinensis White-breasted Nuthatch NULL NULL NULL NULL NULL

In [30]:
ALTER TABLE birds_new
MODIFY COLUMN endangered
ENUM('Extinct',
'Extinct in Wild',
'Threatened - Critically Endangered',
'Threatened - Endangered',
'Threatened - Vulnerable',
'Lower Risk - Conservation Dependent',
'Lower Risk - Near Threatened',
'Lower Risk - Least Concern')
AFTER family_id;

ERROR 1265 (01000): Data truncated for column 'endangered' at row 1


In [32]:
SHOW COLUMNS FROM birds_new LIKE 'endangered'

Field Type Null Key Default Extra endangered bit(1) YES b'1'

In [34]:
UPDATE birds_new
SET endangered = 7;

ERROR 1406 (22001): Data too long for column 'endangered' at row 1


### Dynamic Columns

In [35]:
USE birdwatchers;
CREATE TABLE surveys
(survey_id INT AUTO_INCREMENT KEY,
survey_name VARCHAR(255));
CREATE TABLE survey_questions
(question_id INT AUTO_INCREMENT KEY,
survey_id INT,
question VARCHAR(255),
choices BLOB);
CREATE TABLE survey_answers
(answer_id INT AUTO_INCREMENT KEY,
human_id INT,
question_id INT,
date_answered DATETIME,
answer VARCHAR(255));

In [36]:
INSERT INTO surveys (survey_name)
VALUES("Favorite Birding Location");
INSERT INTO survey_questions
(survey_id, question, choices)
VALUES(LAST_INSERT_ID(),
"What's your favorite setting for bird-watching?",
COLUMN_CREATE('1', 'forest', '2', 'shore', '3', 'backyard') );
INSERT INTO surveys (survey_name)
VALUES("Preferred Birds");
INSERT INTO survey_questions
(survey_id, question, choices)
VALUES(LAST_INSERT_ID(),
"Which type of birds do you like best?",
COLUMN_CREATE('1', 'perching', '2', 'shore', '3', 'fowl', '4', 'rapture') );

In [37]:
SELECT COLUMN_GET(choices, 3 AS CHAR)
AS 'Location'
FROM survey_questions
WHERE survey_id = 1;

Location backyard

In [38]:
INSERT INTO survey_answers
(human_id, question_id, date_answered, answer)
VALUES
(29, 1, NOW(), 2),
(29, 2, NOW(), 2),
(35, 1, NOW(), 1),
(35, 2, NOW(), 1),
(26, 1, NOW(), 2),
(26, 2, NOW(), 1),
(27, 1, NOW(), 2),
(27, 2, NOW(), 4),
(16, 1, NOW(), 3),
(3, 1, NOW(), 1),
(3, 2, NOW(), 1);

In [39]:
SELECT IFNULL(COLUMN_GET(choices, answer AS CHAR), 'total')
AS 'Birding Site', COUNT(*) AS 'Votes'
FROM survey_answers
JOIN survey_questions USING(question_id)
WHERE survey_id = 1
AND question_id = 1
GROUP BY answer WITH ROLLUP;

Birding Site Votes forest 2 shore 3 backyard 1 total 6

### Setting a Column’s Default Value

In [40]:
CREATE TABLE rookery.conservation_status
(status_id INT AUTO_INCREMENT PRIMARY KEY,
conservation_category CHAR(10),
conservation_state CHAR(25) );

In [41]:
INSERT INTO rookery.conservation_status
(conservation_category, conservation_state)
VALUES('Extinct','Extinct'),
('Extinct','Extinct in Wild'),
('Threatened','Critically Endangered'),
('Threatened','Endangered'),
('Threatened','Vulnerable'),
('Lower Risk','Conservation Dependent'),
('Lower Risk','Near Threatened'),
('Lower Risk','Least Concern');

In [42]:
SELECT * FROM rookery.conservation_status;

status_id conservation_category conservation_state 1 Extinct Extinct 2 Extinct Extinct in Wild 3 Threatened Critically Endangered 4 Threatened Endangered 5 Threatened Vulnerable 6 Lower Risk Conservation Dependent 7 Lower Risk Near Threatened 8 Lower Risk Least Concern

In [44]:
ALTER TABLE birds_new
CHANGE COLUMN endangered conservation_status_id INT DEFAULT 8;

ERROR 1146 (42S02): Table 'birdwatchers.birds_new' doesn't exist


In [45]:
ALTER TABLE birds_new
ALTER conservation_status_id SET DEFAULT 7;

ERROR 1146 (42S02): Table 'birdwatchers.birds_new' doesn't exist


In [46]:
statement
only for that column:
SHOW COLUMNS FROM birds_new LIKE 'conservation_status_id';

ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'statement only for that column: SHOW COLUMNS FROM birds_new LIKE 'conservatio...' at line 1


In [48]:
ALTER TABLE birds_new
ALTER conservation_status_id DROP DEFAULT;

ERROR 1146 (42S02): Table 'birdwatchers.birds_new' doesn't exist


### Setting the Value of AUTO_INCREMENT

In [49]:
SELECT auto_increment
FROM information_schema.tables
WHERE table_name = 'birds';

auto_increment 7

In [50]:
USE rookery
ALTER TABLE birds
AUTO_INCREMENT = 10;

### Another Method to Alter and Create a Table

In [51]:
DESCRIBE birds;
DESCRIBE birds_new;
SELECT * FROM birds_new;

Field Type Null Key Default Extra bird_id int(11) NO PRI NULL auto_increment scientific_name varchar(255) YES UNI NULL common_name varchar(50) YES NULL family_id int(11) YES NULL description text YES NULL

ERROR 1146 (42S02): Table 'rookery.birds_new' doesn't exist
ERROR 1146 (42S02): Table 'rookery.birds_new' doesn't exist


In [52]:
SHOW CREATE TABLE birds;

Table Create Table birds CREATE TABLE `birds` (

In [53]:
ALTER TABLE birds_new
AUTO_INCREMENT = 6;

ERROR 1146 (42S02): Table 'rookery.birds_new' doesn't exist


In [54]:
CREATE TABLE birds_details
SELECT bird_id, description
FROM birds;

In [55]:
DESCRIBE birds_details;

Field Type Null Key Default Extra bird_id int(11) NO 0 description text YES NULL

In [56]:
ALTER TABLE birds
DROP COLUMN description;

### Renaming a Table

In [57]:
RENAME TABLE table1_altered
TO table1;

ERROR 1146 (42S02): Table 'rookery.table1_altered' doesn't exist


In [58]:
RENAME TABLE rookery.birds TO rookery.birds_old,
test.birds_new TO rookery.birds;

In [59]:
SHOW TABLES IN rookery LIKE 'birds%';

Tables_in_rookery (birds%) birds birds_details birds_old

In [60]:
DROP TABLE birds_old;

### Reordering a Table

In [61]:
SELECT * FROM country_codes
LIMIT 5;

ERROR 1146 (42S02): Table 'rookery.country_codes' doesn't exist


In [62]:
ALTER TABLE country_codes
ORDER BY country_code;

ERROR 1146 (42S02): Table 'rookery.country_codes' doesn't exist


In [63]:
SELECT * FROM
country_codes LIMIT 5;

ERROR 1146 (42S02): Table 'rookery.country_codes' doesn't exist


In [64]:
SELECT * FROM country_codes
ORDER BY country_name
LIMIT 5;

ERROR 1146 (42S02): Table 'rookery.country_codes' doesn't exist


### Indexes 

In [65]:
ALTER TABLE conservation_status
CHANGE status_id conservation_status_id INT AUTO_INCREMENT PRIMARY KEY;

ERROR 1068 (42000): Multiple primary key defined


In [66]:
SHOW INDEX FROM birdwatchers.humans;

Table Non_unique Key_name Seq_in_index Column_name Collation Cardinality Sub_part Packed Null Index_type Comment Index_comment humans 0 PRIMARY 1 human_id A 0 NULL NULL BTREE

In [67]:
EXPLAIN SELECT * FROM birdwatchers.humans
WHERE name_last = 'Hollar';

id select_type table type possible_keys key key_len ref rows Extra 1 SIMPLE humans ALL NULL NULL NULL NULL 1 Using where

In [68]:
ALTER TABLE birdwatchers.humans
ADD INDEX human_names (name_last, name_first);

In [70]:
SHOW CREATE TABLE birdwatchers.humans;

Table Create Table humans CREATE TABLE `humans` (

In [71]:
SHOW INDEX FROM birdwatchers.humans
WHERE Key_name = 'human_names';

Table Non_unique Key_name Seq_in_index Column_name Collation Cardinality Sub_part Packed Null Index_type Comment Index_comment humans 1 human_names 1 name_last A 0 NULL NULL YES BTREE humans 1 human_names 2 name_first A 0 NULL NULL YES BTREE

In [72]:
EXPLAIN SELECT * FROM birdwatchers.humans
WHERE name_last = 'Hollar';

id select_type table type possible_keys key key_len ref rows Extra 1 SIMPLE humans ref human_names human_names 103 const 1 Using index condition

In [73]:
ALTER TABLE conservation_status
DROP PRIMARY KEY,
CHANGE status_id conservation_status_id INT PRIMARY KEY AUTO_INCREMENT;

### Summary

Good planning is certainly key to developing an efficient database. However, as you can
see from all of the examples of how to use the ALTER TABLE statement, MySQL is malleable
enough that a database and its tables can be reshaped without much trouble. Just
be sure to make a backup before restructuring a database, and work from a copy of a
table before altering it. Check your work and the data when you’re finished, before
committing the changes made.
With all of this in mind, after having had some experience altering tables in this chapter,
you should feel comfortable in creating tables, as you now know that they don’t have to
be perfect from the beginning. You should also have a good sense of the options available
with columns and how to set them. And you should have a basic understanding of
indexes, how they’re used, and how they may be created and changed.
If you have found this chapter confusing, though, it may be that you need more experience
using tables with data. In the next part of this book, you will get plenty of experience
working with tables, inserting data into columns, and changing the data. When
you see how the data comes together, you’ll have a better understanding of how to
structure a table and how to set columns in preparation for data. You’ll have a better
appreciation of how multiple tables may be joined together to get the results you want.